In [2]:
import pandas as pd
import awswrangler as awr
import openpyxl
import os
import shutil
import unicodedata
import win32com.client as win32

#EXTRACT
path_to_query = r"C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista\base_seguro_de_vida.sql"

with open (path_to_query,'r') as reading:
    query_readed = reading.read()

df = awr.athena.read_sql_query(query_readed,'silver')

In [3]:


row = df['e_mail'].isnull() & df['unidade'].str.contains('UNIDADE ')
df.loc[row, 'e_mail'] = 'unidade.' + df.loc[row, 'unidade'].str.replace('UNIDADE ', '').str.lower() + '@transdesk.com.br'

df.head()


,cooperativa,unidade,cliente,conjunto,coverage,beneficio,placa,motorista,e_mail
0,Viavante,UNIDADE ITAJAI,Adilson Hettwer,5781,27690,RASTREADOR,MWX9644,<NA>,unidade.itajai@transdesk.com.br
1,Viavante,UNIDADE ITAJAI,Adilson Hettwer,5776,27690,RASTREADOR,MWX9644,<NA>,unidade.itajai@transdesk.com.br
2,Viavante,UNIDADE ITAJAI,Adilson Hettwer,7113,27689,CASCO (VEÍCULO),MWX9644,<NA>,unidade.itajai@transdesk.com.br
3,Viavante,UNIDADE ITAJAI,Adilson Hettwer,6480,27689,CASCO (VEÍCULO),MWX9644,<NA>,unidade.itajai@transdesk.com.br
4,Viavante,UNIDADE ITAJAI,Adilson Hettwer,6477,27689,CASCO (VEÍCULO),MWX9644,<NA>,unidade.itajai@transdesk.com.br


In [4]:
#removing empty spaces
df['e_mail'] = df['e_mail'].str.replace(' ','')

#removing diacritics with unicodedata

def remove_diacritics (texto):
    if isinstance(texto,str):
        return ''.join(c for c in unicodedata.normalize('NFKD',texto) if not unicodedata.combining(c))
    return texto

df['e_mail'] = df['e_mail'].apply(remove_diacritics)

df = df.reset_index(drop=True)


In [5]:
df.head()

,cooperativa,unidade,cliente,conjunto,coverage,beneficio,placa,motorista,e_mail
0,Viavante,UNIDADE ITAJAI,Adilson Hettwer,5781,27690,RASTREADOR,MWX9644,<NA>,unidade.itajai@transdesk.com.br
1,Viavante,UNIDADE ITAJAI,Adilson Hettwer,5776,27690,RASTREADOR,MWX9644,<NA>,unidade.itajai@transdesk.com.br
2,Viavante,UNIDADE ITAJAI,Adilson Hettwer,7113,27689,CASCO (VEÍCULO),MWX9644,<NA>,unidade.itajai@transdesk.com.br
3,Viavante,UNIDADE ITAJAI,Adilson Hettwer,6480,27689,CASCO (VEÍCULO),MWX9644,<NA>,unidade.itajai@transdesk.com.br
4,Viavante,UNIDADE ITAJAI,Adilson Hettwer,6477,27689,CASCO (VEÍCULO),MWX9644,<NA>,unidade.itajai@transdesk.com.br


Conferência original

In [29]:
df_aggregated = df.groupby(['unidade','beneficio','conjunto','e_mail']).agg(
    placas = ('placa','count'),
    motoristas = ('motorista','count')
).reset_index()


df_aggregated_new = df_aggregated[df_aggregated['placas']>df_aggregated['motoristas']]

df_emails  = pd.DataFrame(df_aggregated_new['e_mail'].dropna().unique(), columns=['e-mails'])

path_to_save = r'C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista'
file_name_emails = 'lista_emails.xlsx'
final_path_email = os.path.join(path_to_save, file_name_emails)



df_emails.to_excel(final_path_email, engine='openpyxl',index=False)





In [34]:
df_aggregated_new

,unidade,beneficio,conjunto,e_mail,placas,motoristas
119,TS CONSULTORIA EM TRANSPORTES LTDA,ASSESSORIA JURÍDICA,5769,contasapagar.unus@gmail.com,1,0
120,TS CONSULTORIA EM TRANSPORTES LTDA,ASSESSORIA JURÍDICA,6317,contasapagar.unus@gmail.com,1,0
121,TS CONSULTORIA EM TRANSPORTES LTDA,ASSESSORIA JURÍDICA,8332,contasapagar.unus@gmail.com,1,0
122,TS CONSULTORIA EM TRANSPORTES LTDA,ASSESSORIA JURÍDICA,8513,contasapagar.unus@gmail.com,1,0
123,TS CONSULTORIA EM TRANSPORTES LTDA,ASSESSORIA JURÍDICA,8706,contasapagar.unus@gmail.com,1,0
...,...,...,...,...,...,...
576982,UNIDADE VILHENA,VIDROS,134058,maria.marchezi@transdesk.com.br,1,0
576983,UNIDADE VILHENA,VIDROS,134166,maria.marchezi@transdesk.com.br,1,0
576984,UNIDADE VILHENA,VIDROS,134931,maria.marchezi@transdesk.com.br,1,0
576985,UNIDADE VILHENA,VIDROS,135033,maria.marchezi@transdesk.com.br,1,0


Dataframe de conferência agregada de e-mail

In [54]:
df_aggregated_unidemail = df.groupby(['unidade', 'e_mail']).agg(
    placas=('placa', 'count'),
    motoristas=('motorista', 'count')
).reset_index()

df_numeros = df_aggregated_unidemail.groupby(['unidade']).agg(
    qtd_placas=('placas', 'sum'),
    qtd_motoristas=('motoristas', 'sum')
).reset_index()

for idx, row in df_aggregated_unidemail.iterrows():
    iEmail = row['e_mail']
    iUnidade = row['unidade']


    unidade_info = df_numeros.loc[df_numeros['unidade'] == iUnidade].iloc[0]

    iQtdPlacas = unidade_info['qtd_placas']
    iQtdMotoristas = unidade_info['qtd_motoristas']

    print(f"Prezado {iEmail}, sua unidade:{iUnidade} tem {iQtdPlacas} placas e {iQtdMotoristas} motoristas.")

Prezado n@n.n, sua unidade:COOPERATIVA DE TRABALHO DOS PROP. DE VEIC. CARGAS - SEGTRUCK tem 0 placas e 0 motoristas.
Prezado contasapagar.unus@gmail.com, sua unidade:TS CONSULTORIA EM TRANSPORTES LTDA tem 935 placas e 218 motoristas.
Prezado jairo.torres@transdesk.com.br, sua unidade:UNIDADE ARACAJU tem 14855 placas e 1558 motoristas.
Prezado maria.olga@transdesk.com.br, sua unidade:UNIDADE ARACAJU tem 14855 placas e 1558 motoristas.
Prezado unidade.aracaju@transdesk.com.br, sua unidade:UNIDADE ARACAJU tem 14855 placas e 1558 motoristas.
Prezado unidade.campogrande@transdesk.com.br, sua unidade:UNIDADE CAMPO GRANDE tem 1764 placas e 88 motoristas.
Prezado rafael.sartori@transdesk.com.br, sua unidade:UNIDADE CARAZINHO tem 1575 placas e 158 motoristas.
Prezado unidade.carazinho@transdesk.com.br, sua unidade:UNIDADE CARAZINHO tem 1575 placas e 158 motoristas.
Prezado moises.junior@transdesk.com.br, sua unidade:UNIDADE CASCAVEL tem 38253 placas e 2716 motoristas.
Prezado unidade.cascavel@t

In [33]:
df_emails

,e-mails
0,contasapagar.unus@gmail.com
1,unidade.aracaju@transdesk.com.br
2,jairo.torres@transdesk.com.br
3,maria.olga@transdesk.com.br
4,unidade.campogrande@transdesk.com.br
5,unidade.carazinho@transdesk.com.br
6,rafael.sartori@transdesk.com.br
7,moises.junior@transdesk.com.br
8,unidade.cascavel@transdesk.com.br
9,unidade.chapeco@transdesk.com.br


In [50]:
df_aggregated_unidemail

,unidade,e_mail,placas,motoristas
0,COOPERATIVA DE TRABALHO DOS PROP. DE VEIC. CAR...,n@n.n,0,0
1,TS CONSULTORIA EM TRANSPORTES LTDA,contasapagar.unus@gmail.com,935,218
2,UNIDADE ARACAJU,jairo.torres@transdesk.com.br,7129,723
3,UNIDADE ARACAJU,maria.olga@transdesk.com.br,7129,723
4,UNIDADE ARACAJU,unidade.aracaju@transdesk.com.br,597,112
...,...,...,...,...
56,UNIDADE SÃO PAULO,unidade.saopaulo@transdesk.com.br,2902,12
57,UNIDADE UBERLÂNDIA,,145,13
58,UNIDADE UBERLÂNDIA,unidade.uberlandia@transdesk.com.br,2195,148
59,UNIDADE VILHENA,maria.marchezi@transdesk.com.br,14409,1150


FOR do e-mail

In [ ]:
for index, iContato in enumerate(df['e_mail'].unique()):



    
    enviar_email()

Automação e-mail

In [32]:

def enviar_email():    
    outlook = win32.Dispatch('outlook.application')  #integração do outlook com o python

    #criar e-mail
    email = outlook.CreateItem(0)

    #configurar as informações do e-mail
    email.To = iContato
    email.Subject = "Teste1"
    email.HTMLBody =f"""
    Boa tarde,

    Sua unidade {iUnidade} falta cadastrar 
 
    """

    email.Send()

    print("Email enviado")

Carregando para as pastas

In [ ]:
#LOAD

path_to_save = r'C:\Users\raphael.almeida\Documents\Projetos\Seguro de Vida Motorista'
file_name = 'Cobertura de Seguros Motoristas.xlsx'
final_path = os.path.join(path_to_save, file_name)
os.remove(final_path)
df.to_excel(final_path, engine='openpyxl', index = False)